# Lab 08: AI Vulnerability Scanner

Build an AI-enhanced vulnerability scanner with intelligent prioritization.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/depalmar/ai_for_the_win/blob/main/notebooks/lab08_vuln_scanner_ai.ipynb)

## Learning Objectives
- CVE analysis and prioritization
- CVSS scoring interpretation
- Exploitability assessment
- AI-powered remediation guidance

In [ ]:
import json
from typing import List, Dict
from dataclasses import dataclass
from enum import Enum

## 1. Vulnerability Data Model

In [ ]:
class Severity(Enum):
    CRITICAL = 4
    HIGH = 3
    MEDIUM = 2
    LOW = 1
    INFO = 0

@dataclass
class Vulnerability:
    cve_id: str
    title: str
    cvss_score: float
    severity: Severity
    description: str
    affected_product: str
    affected_version: str
    exploit_available: bool
    patch_available: bool
    
    def to_dict(self) -> Dict:
        return {
            "cve_id": self.cve_id,
            "title": self.title,
            "cvss_score": self.cvss_score,
            "severity": self.severity.name,
            "description": self.description,
            "affected_product": self.affected_product,
            "exploit_available": self.exploit_available,
            "patch_available": self.patch_available
        }

# Sample vulnerabilities
SAMPLE_VULNS = [
    Vulnerability(
        cve_id="CVE-2024-0001",
        title="Remote Code Execution in Apache",
        cvss_score=9.8,
        severity=Severity.CRITICAL,
        description="Unauthenticated RCE via path traversal",
        affected_product="Apache HTTP Server",
        affected_version="< 2.4.52",
        exploit_available=True,
        patch_available=True
    ),
    Vulnerability(
        cve_id="CVE-2024-0002",
        title="SQL Injection in Web App",
        cvss_score=8.5,
        severity=Severity.HIGH,
        description="Authenticated SQLi in search function",
        affected_product="Custom Web App",
        affected_version="1.0.0",
        exploit_available=False,
        patch_available=False
    ),
    Vulnerability(
        cve_id="CVE-2024-0003",
        title="Information Disclosure",
        cvss_score=5.3,
        severity=Severity.MEDIUM,
        description="Verbose error messages expose internal paths",
        affected_product="Custom Web App",
        affected_version="1.0.0",
        exploit_available=False,
        patch_available=True
    )
]

print(f"Loaded {len(SAMPLE_VULNS)} vulnerabilities")

## 2. Risk Prioritization

In [ ]:
class VulnPrioritizer:
    """Prioritize vulnerabilities based on risk factors."""
    
    def calculate_risk_score(self, vuln: Vulnerability) -> float:
        """Calculate composite risk score."""
        score = vuln.cvss_score
        
        # Boost for exploit availability
        if vuln.exploit_available:
            score += 1.0
        
        # Reduce for patch availability
        if vuln.patch_available:
            score -= 0.5
        
        return min(10.0, max(0.0, score))
    
    def prioritize(self, vulns: List[Vulnerability]) -> List[Dict]:
        """Sort vulnerabilities by risk."""
        scored = []
        for v in vulns:
            risk = self.calculate_risk_score(v)
            scored.append({
                "vulnerability": v,
                "risk_score": risk,
                "priority": "P1" if risk >= 9 else "P2" if risk >= 7 else "P3"
            })
        
        return sorted(scored, key=lambda x: x["risk_score"], reverse=True)

# Prioritize
prioritizer = VulnPrioritizer()
prioritized = prioritizer.prioritize(SAMPLE_VULNS)

print("Prioritized Vulnerabilities:")
print("=" * 60)
for item in prioritized:
    v = item["vulnerability"]
    print(f"[{item['priority']}] {v.cve_id} - Risk: {item['risk_score']:.1f}")
    print(f"    {v.title}")
    print(f"    Exploit: {'Yes' if v.exploit_available else 'No'} | Patch: {'Yes' if v.patch_available else 'No'}")
    print()

## 3. AI Remediation Advisor

In [ ]:
class RemediationAdvisor:
    """AI-powered remediation guidance."""
    
    def __init__(self):
        try:
            from anthropic import Anthropic
            self.client = Anthropic()
            self.available = True
        except:
            self.available = False
    
    def get_remediation(self, vuln: Vulnerability) -> str:
        """Get remediation advice for vulnerability."""
        if not self.available:
            return self._mock_remediation(vuln)
        
        prompt = f"""Provide remediation guidance for this vulnerability:

CVE: {vuln.cve_id}
Title: {vuln.title}
CVSS: {vuln.cvss_score}
Description: {vuln.description}
Product: {vuln.affected_product} {vuln.affected_version}
Exploit Available: {vuln.exploit_available}
Patch Available: {vuln.patch_available}

Provide:
1. Immediate mitigation steps
2. Long-term remediation
3. Detection recommendations

Be specific and actionable."""
        
        response = self.client.messages.create(
            model="claude-sonnet-4-20250514",
            max_tokens=512,
            messages=[{"role": "user", "content": prompt}]
        )
        
        return response.content[0].text
    
    def _mock_remediation(self, vuln: Vulnerability) -> str:
        return f"""
## Remediation for {vuln.cve_id}

### Immediate Mitigations
1. Apply vendor patch if available
2. Implement WAF rules to block exploitation
3. Restrict network access to affected systems

### Long-term Remediation
1. Update {vuln.affected_product} to latest version
2. Review and harden configuration
3. Implement defense-in-depth controls

### Detection
- Monitor for exploitation attempts in logs
- Enable IDS/IPS signatures for {vuln.cve_id}
- Set up alerts for suspicious activity
"""

# Get advice
advisor = RemediationAdvisor()
advice = advisor.get_remediation(SAMPLE_VULNS[0])
print(advice)

## 4. Scan Report Generator

In [ ]:
def generate_scan_report(vulns: List[Vulnerability]) -> str:
    """Generate vulnerability scan report."""
    prioritizer = VulnPrioritizer()
    prioritized = prioritizer.prioritize(vulns)
    
    # Statistics
    stats = {
        "total": len(vulns),
        "critical": sum(1 for v in vulns if v.severity == Severity.CRITICAL),
        "high": sum(1 for v in vulns if v.severity == Severity.HIGH),
        "exploitable": sum(1 for v in vulns if v.exploit_available)
    }
    
    report = f"""
# Vulnerability Scan Report

## Executive Summary
- Total Vulnerabilities: {stats['total']}
- Critical: {stats['critical']}
- High: {stats['high']}
- Exploitable: {stats['exploitable']}

## Prioritized Findings
"""
    
    for item in prioritized:
        v = item["vulnerability"]
        report += f"""
### [{item['priority']}] {v.cve_id} - {v.title}
- **CVSS**: {v.cvss_score} ({v.severity.name})
- **Risk Score**: {item['risk_score']:.1f}
- **Product**: {v.affected_product} {v.affected_version}
- **Exploit Available**: {'Yes' if v.exploit_available else 'No'}
- **Patch Available**: {'Yes' if v.patch_available else 'No'}

{v.description}
"""
    
    return report

# Generate report
report = generate_scan_report(SAMPLE_VULNS)
print(report)

## Summary

We built an AI-enhanced vulnerability scanner:

1. **Data Model** - Structured vulnerability representation
2. **Prioritization** - Risk-based scoring algorithm
3. **AI Advisor** - LLM-powered remediation guidance
4. **Reporting** - Automated scan reports

### Next Steps:
1. Integrate with CVE databases (NVD API)
2. Add asset context for prioritization
3. Create automated remediation workflows